# O Motorista Impaciente
<img src="Figuras\chaplinImpaciente.gif" alt="Drawing" style="width: 300px;"/>


## Introdução
Considere o problema do autocarro da avaliação contínua 1 em que um motorista de autocarro com capacidade limitada, sai de uma estação e vai apanhar e largar passageiros de acordo com as reservas, regressando à estação. Ele conhece o mapa da região que corresponde a uma grelha quadrada, sabendo quais as ruas pedonais, quais as de sentido único e o tipo de tráfego em cada rua. Está a par também do tempo médio que leva a atravessar cada rua, que pode ser de tráfego ligeiro, médio ou intenso e tem uma estimativa do tempo que leva cada passageiro a entrar e a sair nas paragens. O seu objectivo é planear o trajecto que leva menos tempo.

O motorista de autocarro é um faz tudo, faz lembrar o funcionário do cinema que vende bilhetes e pipocas, senta as pessoas, projecta o filme, limpa o cinema e fecha a porta. Ele não só tem de tratar das reservas, como planear as viagens e conduzir o autocarro. Ele sabe que nos problemas mais complexos, planear de uma forma óptima exige bastante tempo, embora resulte em tempos de viagens mais curtos e menos gasto de combustível. Para piorar, problemas complexos aparecem todos os dias. No entanto, o motorista detesta planear e adora conduzir, e prefere assim fazer rapidamente um plano "impaciente", de trajectos para lá do óptimo. que o torna mais feliz.

## Tem o motorista motivos para a impaciência?
<img src="Figuras\waiting.gif" alt="Drawing" style="width: 250px;"/>

Para ajudar a compreender porque o motorista detesta fazer planos quinquenais, Vejam o exemplo seguinte:


In [2]:
from elEstadoMundo import *
from oBus import *

In [3]:
capacidade=5
estacao=(1,2)
pedonais={((4,3),(4,4))}
sentidoUnico={((3,3),(3,2)),((1,2),(2,2))}
intenso={((3,2),(3,3)),(3,1),(3,2),((2,0),(2,1))}
ligeiro={((2,1),(2,2))}
tempos={'ligeiro': 50, 'médio': 300, 'intenso': 1000}
tempoPassageiro=10
clientes={'Valdemar':((3,2),(20,20)),'Francis':((10,10),(4,4)),'Berta':((0,0),(7,0)),'Zulu':((1,3),(2,2))}

mundoX=MundoBus(25,estacao,capacidade,pedonais,sentidoUnico,intenso,ligeiro,tempos,tempoPassageiro,clientes)

p=OBus(mundo=mundoX)

Vamos aplicar o Custo Uniforme, que é o algoritmo que o motorista utiliza, porque a sua mente é pouco heurística.

In [5]:
res_best = uniform_cost_search(p)
print(p.prettyPlan(res_best.solution()))
print(res_best.path_cost)

['n', ([], ['Zulu']), 's', 'e', (['Zulu'], []), 'e', ([], ['Valdemar']), 'o', 's', 'o', 'o', 's', ([], ['Berta']), 'e', 'e', 'e', 'e', 'e', 'e', 'e', (['Berta'], []), 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'e', 'e', 'e', ([], ['Francis']), 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', (['Valdemar'], []), 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's', (['Francis'], []), 'o', 'o', 'o', 's', 's']
25630


## Planeamento Impaciente

O motorista vai ter de construir um plano global para partir da estação, apanhar e largar todos os clientes e regressar à estação. Mas, como dissemos em cima, esse plano não tem de ser óptimo e correponde à divisão de um problem em subproblemas e à construção de uma sucessão de planos impacientes, mais simples, esses sim óptimos em termos de objectivos de mais curto prazo. 

Um subproblema mais simples consiste em encontrar apenas a melhor trajectória até ao lugar mais próximo relevante, i.e., que pode ser a estação quando o autocarro está vazio ou a localização de um ou mais clientes ou o lugar de saída de um passageiro ou mais passageiros. Depois de construir o plano óptimo, solução de cada subproblema, o condutor poderá incluir no plano a acção de largar e/ou apanhar passageiros e ataca o subproblema seguinte, até completar o plano.

Inicialmente, a partir da estação, ele pretende apanhar os clientes mais próximos e planeia o melhor trajecto para satisfazer esse primeiro objectivo. Tratará de apanhar e/ou largar passageiros nesse lugar e, a seguir, ele terá como objectivo largar ou apanhar os clientes no destino ou localização mais próxima, e assim sucessivamente até largar os últimos passageiros. Aí, irá finalmente considerar o melhor plano para regressar à estação. O plano final corresponderá à concatenação dos planos impacientes parciais, que são impacientes precisamente porque ele deseja sempre planear o melhor trajecto até ao objectivo mais próximo.

Notem que, para simplificar, vamos considerar que o autocarro tem sempre capacidade suficiente para caberem todos os clientes. No fundo é como se a sua capacidade fosse ilimitada. 


## ZOOM IN sobre o Replaneamento Impaciente
<img src="Figuras\zoomVanGoghIkea.gif" alt="Drawing" style="width: 350px;"/>

O motorista não liga nenhuma ao tráfego ou aos sentidos das ruas quando decide qual o próximo objectivo a planear, apenas toma em consideração a distância ortogonal aos pontos da grelha que são candidatos a serem o próximo objectivo, utilizando a distância de Manhatan. O condutor não está para complicações e em caso de empate, considera o primeiro ponto por ordem crescente dos empatados e que está de acordo com a ordem dos tuplos: primeiro ordenam-se em relação ao primeiro elemento (eixo dos $Xs$) e depois em relação ao segundo elemento (eixo dos $Ys$).  Assim se os dois pontos interessantes mais próximos forem $(2,2)$ e $(1,2)$ ele preferirá o segundo e se forem $(1,2)$ e $(1,3)$ ele preferirá o primeiro.

Então vejamos com mais detalhe como planeia o motorista impaciente:

1. O motorista começa por ter como objectivo ir apanhar os clientes mais próximos. Para isso planeia o trajecto mais rápido entre a estação e a localização dos clientes que esteja mais próxima. Inicia o plano final com a trajectória criada.

2. Apanha os clientes que estão na posição actual, i.e. junta essas acções ao plano final.

3. Enquanto houver passageiros para largar ou pessoas para apanhar, define como objectivo seguinte ir da maneira mais rápida até ao lugar mais próximo entre os destinos dos que estão no autocarro e as localizações dos que estão ainda lá fora. Planeado o trajecto óptimo, adiciona-o ao plano final juntamente com as acções de largada e/ou entrada de pessoas no lugar corrente.

4. Com o autocarro vazio, o plano agora é o melhor trajecto de regresso à estação que completa o plano final.


Como vimos, o plano final é a concatenação dos planos óptimos parciais, separados pelas acções de entrada/saida de pessoas do autocarro, juntamente com o tempo correspondendo à soma dos custos dos vários planos. 

## Objetivo
Desenvolver a função em Python: `planoImpaciente`, que recebe uma instância da classe MundoBus e devolve a lista de planos impacientes.
<img src="Figuras\JapaneseBus4.gif" alt="Drawing" style="width: 250px;"/>

## A Função `planoImpaciente`
...recebe uma instância da classe `MundoBus` e o plano impaciente final juntamente com a sua duração total. O plano final será uma sequências de acções de movimento: "s","n","e" e "o" e de apanhar/largar clientes na sua versão pretty, i.e., os conjuntos formados pelos identificadores dos clientes passam a listas ordenadas de forma crescente dos identificadores. Notem que a razão porque passamos conjuntos a listas ordenadas, é porque precisamos de fazer a correcção automática e o proint dos conjuntos é pouco rigoroso. Notem que embora `MundoBus` indique a capacidade do autocarro, esse valor é completamente ignorado para a geração do plano final, porque assumimos que o autocarro tem sempre capaidade infinita. 

Consideremos alguns exemplos:

### Exemplo 1
<img src="Figuras\japaneseBus.gif" alt="Drawing" style="width: 350px;"/>

Um dia tranquilo para o motorista, onde só há uma reserva.

``` python
dimensao=20
estacao=(0,0)
capacidade=2
pedonais={((2,2),(2,3)),((2,2),(3,2)),((1,2),(2,2)),((2,1),(2,2))}
sentidoUnico={((5,1),(5,0)),((4,0),(5,0))}
intenso={((0,0),(1,0)),((1,0),(2,0))}
ligeiro={((2,3),(2,4))}
tempos={'ligeiro': 200, 'médio': 500, 'intenso': 2000}
tempoPassageiro=10
clientes={1:((1,0),(2,0))}
mundo1=MundoBus(dimensao,estacao,capacidade,pedonais,sentidoUnico,intenso,ligeiro,tempos,tempoPassageiro,clientes)

```

Neste caso, o que faz o motorista *Impaciente* é:
* planear o melhor caminho até ao cliente, que é ir para norte, depois leste e depois para sul, porque a rua $(0,0)\rightarrow(1,0)$ é de tráfego intenso, com custo $1500s$,
* depois apanha o cliente, com custo $10s$,
* planeia o melhor caminho até o destino do cliente, que devido à rua $(1,0)\rightarrow(2,0)$ ser de tráfego intenso, o obriga a ir para norte, depois leste e sul, que custa $1500s$,
* larga-o, custando mais $10s$,
* e regressa pelo melhor caminho, que é indo por norte, depois oeste, de novo oeste e finalmente sul, com duração de $2000s$.



Para testar no python este exemplo faremos:

```python
planoImpaciente(mundo1)
(['n', 'e', 's', ([], [1]), 'n', 'e', 's', ([1], []), 'n', 'o', 'o', 's'], 5020)

```
Note que o plano ideal, paciente, seria também este, porque só há um cliente, mas isso não acontecerá sempre, muito pelo contrário.

### Exemplo 2
Nesta situação, temos 4 reservas:
``` python
dimensao=10
estacao=(0,0)
capacidade=20
pedonais={((2,2),(2,3)),((2,2),(3,2)),((1,3),(1,4))}
sentidoUnico={((2,1),(2,2)),((3,1),(3,2)),((3,2),(3,3))}
intenso={((1,2),(1,3))}
ligeiro={((0,2),(0,3)),((0,3),(1,3))}
tempos={'ligeiro': 200, 'médio': 500, 'intenso': 1500}
tempoPassageiro=10
clientes={'A':((1,0),(3,0)),'B':((2,0),(4,0)),'C':((3,0),(5,0)),'D':((0,2),(5,0))}

mundo2=MundoBus(dimensao,estacao,capacidade,pedonais,sentidoUnico,intenso,ligeiro,tempos,tempoPassageiro,clientes)
```

* Dado que o autocarro está na estação, em $(0,0)$, o alvo mais próximo é o cliente A. Planeia o melhor trajecto até $(1,0)$, que é só ir para este. Custo = $500s$, porque é uma rua de intensidade média de tráfego.
* Está em $(1,0)$ e apanha o cliente A. Custo = $510s$.
* O alvo é agora ir apanhar o B em $(2,0)$. O plano (a trajectória mais rápida) é de novo ir para este. Custo = $1010s$.
* Está em $(2,0)$ e apanha o B. Custo = $1020s$.
* O objectivo é agora largar o A e apanhar o C (ir para $(3,0)$). Planeia o melhor trajecto, que é ir para este. Custo = $1520s$.
* Está em $(3,0)$ e apanha o C depois de largar o A. Custo = $1540s$.
* O alvo é agora largar o B (ir para $(4,0)$). Planeia o melhor trajecto, que é ir para este. Custo = $2040s$.
* Larga B. O custo está em $2050s$.
* Novo objectivo: largar C. Plano: ir para este, com custo $2550s$.
* Largar C, com custo $2560s$.
* Objectivo `Greedy`: ir apanhar D. Plano: oeste cinco vezes e depois duas vezes norte, com custo acumulado de $6060s$.
* Apanha D, com custo $6070s$.
* Novo destino: largar D. Plano: 2 vezes sul e este cinco vezes. Custo total: $9570s$.
* Larga D, com custo acumulado de $9580s$.
* Objectivo: regressar à estação, através de 5 oestes consecutivos, com custo $12080s$.

Para testar no python este exemplo faremos:

```python
planoImpaciente(mundo2)
(['e',
  ([], ['A']),
  'e',
  ([], ['B']),
  'e',
  (['A'], ['C']),
  'e',
  (['B'], []),
  'e',
  (['C'], []),
  'o', 'o', 'o', 'o', 'o', 'n', 'n',
  ([], ['D']),
  's', 's', 'e', 'e', 'e', 'e', 'e',
  (['D'], []),
  'o', 'o', 'o', 'o', 'o'],
 12080)
``` 

<img src="Figuras\japaneseBus3.gif" alt="Drawing" style="width: 350px;"/>

Notem que o plano ideal de um condutor sem as limitações do nosso condutor *Greedy* seria bastante mais rápido, já que começaria por apanhar C:
```
norte
norte
apanhar Dul
sul
este
apanhar A
este
apanhar B
este
largar A e apanhar C
este
largar B
este
largar C e D
oeste
oeste
oeste
oeste
oeste
```

com custo $7080s$



### Exemplo 3
<img src="Figuras\japaneseBus2.gif" alt="Drawing" style="width: 350px;"/>

Este exemplo é exactamente igual ao exemplo que usámos para justificar a impaciência do motorista. Podem confirmar que é muito mais rápido a ser gerado e que tem uma duração claramente maior

```python
capacidade=2
estacao=(1,2)
pedonais={((4,3),(4,4))}
sentidoUnico={((3,3),(3,2)),((1,2),(2,2))}
intenso={((3,2),(3,3)),(3,1),(3,2),((2,0),(2,1))}
ligeiro={((2,1),(2,2))}
tempos={'ligeiro': 50, 'médio': 300, 'intenso': 1000}
tempoPassageiro=10
clientes={'Valdemar':((3,2),(20,20)),'Francis':((10,10),(4,4)),'Berta':((0,0),(7,0)),'Zulu':((1,3),(2,2))}

mundo3=MundoBus(dimensao,estacao,capacidade,pedonais,sentidoUnico,intenso,ligeiro,tempos,tempoPassageiro,clientes)

planoImpaciente(mundo1)
```

```python
(['n',
  ([], ['Zulu']),
  's',
  'e',
  (['Zulu'], []),
  'e',
  ([], ['Valdemar']),
  'o',
  's',
  'o',
  'o',
  's',
  ([], ['Berta']),
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  (['Berta'], []),
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'e',
  'e',
  'e',
  ([], ['Francis']),
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  's',
  's',
  's',
  's',
  's',
  's',
  (['Francis'], []),
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'n',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  'e',
  (['Valdemar'], []),
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  'o',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's',
  's'],
 32830)
```

## Nota
A função de replaneamento é avaliada em $0.625$

## Ficheiros anexos e imports
O ficheiro `elEstadoMundo.py` será importado, juntamente com `utils.py` e `searchPlus.py`, os quais não devem ser alterados. Podem importar os módulos que desejarem.

## Deadline
Esta avaliação contínua é **individual** e terá **correcção automática** sendo submetida através de um quiz na página da disciplina. Para essa correcção automática iremos disponibilizar alguns testes que serão visíveis e outros testes invisíveis. Podem fazer *check* e *submit* as vezes que quiserem, sem quaisquer penalizações, e a nota final será a da tentativa mais elevada sendo a última tentativa submetida automaticamente no fecho do Quiz.

O prazo é Terça-feira, $1 \space de \space Novembro,\space às\space 23:59$.

<img src="Figuras\russia-bus-disguise-02-aptn-jef-181114_hpEmbed_4x3_992.JPG" alt="Drawing" style="width: 350px;"/>

https://www.ndtv.com/offbeat/4-men-dressed-up-as-a-bus-to-cross-vehicle-only-bridge-video-will-make-you-lol-1947013